In [3]:
import numpy as np

# --- 1. 활성화 함수 및 도함수 ---
def tanh(x):
    """tanh 활성화 함수"""
    return np.tanh(x)

def dtanh(tanh_x):
    """tanh 함수의 도함수. 입력은 이미 tanh(x)가 적용된 값이어야 함."""
    return 1 - tanh_x**2

# --- 2. 손실 함수 (Mean Squared Error - MSE) ---
def mse_loss(y_true, y_pred):
    D = y_true.shape[1] # 출력 벡터의 차원 수
    if D == 0:
        return 0
    squared_errors_sum = np.sum((y_pred - y_true)**2)
    mse = squared_errors_sum / D
    return 0.5 * mse # 0.5는 선택 사항

# --- 3. SimpleRNN 직접 구현 클래스 ---
class SimpleRNNNumpy:
    def __init__(self, vocab_size, hidden_size, output_size, learning_rate=0.01):
        self.vocab_size = vocab_size  # 입력 차원 (원-핫 벡터 크기)
        self.hidden_size = hidden_size
        self.output_size = output_size # 출력 차원 (예측하려는 원-핫 벡터 크기)
        self.learning_rate = learning_rate

        # 가중치 초기화 (작은 랜덤 값으로)
        # 입력 -> 은닉층 가중치
        self.Wxh = np.random.randn(vocab_size, hidden_size) * 0.01
        # 이전 은닉 상태 -> 현재 은닉 상태 가중치 (이것이 바로 '순환 가중치'입니다!)
        self.Whh = np.random.randn(hidden_size, hidden_size) * 0.01
        # 은닉층 -> 출력층 가중치
        self.Why = np.random.randn(hidden_size, output_size) * 0.01
        
        # 편향 초기화 (0으로)
        self.bh = np.zeros((1, hidden_size))  # 은닉층 편향
        self.by = np.zeros((1, output_size))  # 출력층 편향

        # 역전파 시 중간 계산값 저장을 위한 딕셔너리
        self.cache = {}

    def forward(self, inputs_sequence):
        """
        순전파를 수행합니다.
        inputs_sequence: 타임스텝별 입력 벡터의 리스트 (예: [[1,0,0], [0,1,0], ...])
        """
        T = len(inputs_sequence)  # 시퀀스 길이
        
        # 중간값 저장을 위해 초기화
        self.cache['x'] = {}  # 각 타임스텝의 입력
        self.cache['h_linear'] = {} # 은닉층 활성화 전 값 (Wx*x + Wh*h_prev + b)
        self.cache['h'] = {0: np.zeros((1, self.hidden_size))} # 초기 은닉 상태 h_0
        self.cache['y_pred'] = {} # 각 타임스텝의 예측값

        outputs_sequence_pred = [] # 최종 예측값들을 저장할 리스트

        for t in range(T):
            xt = np.array(inputs_sequence[t]).reshape(1, -1) # 현재 타임스텝 입력 (1, vocab_size)
            ht_prev = self.cache['h'][t] # 이전 타임스텝의 은닉 상태

            self.cache['x'][t] = xt

            # 은닉 상태 계산: h_t = tanh(x_t @ Wxh + h_{t-1} @ Whh + b_h)
            ht_linear = np.dot(xt, self.Wxh) + np.dot(ht_prev, self.Whh) + self.bh
            ht = tanh(ht_linear)
            
            self.cache['h_linear'][t] = ht_linear
            self.cache['h'][t+1] = ht # 다음 계산을 위해 저장 (t+1 인덱스 사용 주의)

            # 출력 계산: y_t = h_t @ Why + b_y (여기서는 활성화 함수 없이 선형 출력)
            yt_pred = np.dot(ht, self.Why) + self.by
            self.cache['y_pred'][t] = yt_pred
            outputs_sequence_pred.append(yt_pred)
            
        return outputs_sequence_pred

    def backward(self, inputs_sequence, targets_sequence, outputs_sequence_pred):
        """
        역전파 (BPTT - Backpropagation Through Time)를 수행합니다.
        """
        T = len(inputs_sequence)
        
        # 그래디언트 변수들 초기화 (가중치와 동일한 크기로)
        dWxh, dWhh, dWhy = np.zeros_like(self.Wxh), np.zeros_like(self.Whh), np.zeros_like(self.Why)
        dbh, dby = np.zeros_like(self.bh), np.zeros_like(self.by)
        
        # 다음 타임스텝으로부터 전달될 은닉 상태의 그래디언트 (초기에는 0)
        dh_next = np.zeros((1, self.hidden_size)) 
        
        total_loss_for_sequence = 0

        # 시퀀스의 마지막부터 처음까지 역방향으로 순회
        for t in reversed(range(T)):
            xt = self.cache['x'][t]
            ht = self.cache['h'][t+1]       # 순전파 시 t+1 인덱스에 저장된 h_t
            ht_prev = self.cache['h'][t] # h_{t-1}
            # ht_linear = self.cache['h_linear'][t] # h_t 계산 시 tanh 이전 값

            yt_pred = self.cache['y_pred'][t]
            yt_true = np.array(targets_sequence[t]).reshape(1, -1)

            # 현재 타임스텝의 손실 계산 (정보용)
            loss_t = mse_loss(yt_true, yt_pred)
            total_loss_for_sequence += loss_t
            
            # --- 출력층의 그래디언트 계산 ---
            # 손실 함수(MSE)의 예측값 yt_pred에 대한 도함수: (yt_pred - yt_true)
            # (mse_loss에서 0.5를 곱했으므로, 미분 시 0.5 * 2 * (y_pred - y_true) = (y_pred - y_true))
            # 평균을 취했으므로, yt_true.shape[0]로 나눠줌 (여기서는 1)
            dy_pred = (yt_pred - yt_true) / yt_true.shape[0]

            # Why와 by에 대한 그래디언트 누적
            dWhy += np.dot(ht.T, dy_pred)
            dby += dy_pred
            
            # --- 은닉층의 그래디언트 계산 (BPTT의 핵심) ---
            # 현재 은닉 상태 ht에 대한 그래디언트
            # 1. 출력층으로부터 오는 그래디언트: dy_pred @ Why.T
            # 2. 다음 타임스텝 t+1의 은닉 상태로부터 오는 그래디언트: dh_next
            dh = np.dot(dy_pred, self.Why.T) + dh_next
            
            # tanh 활성화 함수의 그래디언트 적용
            # d(loss)/d(ht_linear) = d(loss)/d(ht) * d(ht)/d(ht_linear)
            # d(ht)/d(ht_linear)는 dtanh(ht) 즉, (1 - ht**2)
            dh_raw = dh * dtanh(ht) # dtanh는 이미 tanh가 적용된 ht를 인자로 받음
            
            # bh에 대한 그래디언트 누적
            dbh += dh_raw
            
            # Wxh에 대한 그래디언트 누적
            dWxh += np.dot(xt.T, dh_raw)
            
            # Whh (순환 가중치)에 대한 그래디언트 누적! 이것이 핵심입니다.
            dWhh += np.dot(ht_prev.T, dh_raw)
            
            # 다음 반복(t-1)을 위해, 현재 dh_raw가 이전 은닉 상태 ht_prev에 미치는 영향(dh_next_for_prev_step)을 계산
            dh_next = np.dot(dh_raw, self.Whh.T)
            
        # (선택적) 그래디언트 클리핑: 그래디언트 폭주를 막기 위해 사용
        # 여기서는 매우 간단한 형태로 구현하거나, 값의 범위를 보고 생략 가능
        clip_value = 5.0
        for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
            np.clip(dparam, -clip_value, clip_value, out=dparam)
            
        return total_loss_for_sequence, dWxh, dWhh, dWhy, dbh, dby

    def update_weights(self, dWxh, dWhh, dWhy, dbh, dby):
        """단순 경사 하강법으로 가중치를 업데이트합니다."""
        self.Wxh -= self.learning_rate * dWxh
        self.Whh -= self.learning_rate * dWhh # 순환 가중치 업데이트!
        self.Why -= self.learning_rate * dWhy
        self.bh  -= self.learning_rate * dbh
        self.by  -= self.learning_rate * dby

# --- 4. 더미 데이터 생성 및 학습 루프 ---

# 하이퍼파라미터
vocab_size = 5    # 입력/출력 단어(심볼)의 종류 수
hidden_size = 4   # 은닉 상태의 크기 (자유롭게 설정)
output_size = vocab_size # 다음 심볼을 예측하므로 vocab_size와 동일
learning_rate = 0.1
epochs = 100

# 더미 시퀀스 데이터: 0 -> 1 -> 2 -> 0 (순환)
# 입력: [0, 1, 2], 타겟: [1, 2, 0] (다음 스텝 예측)
# 원-핫 인코딩으로 표현
# x_0 = [1,0,0], x_1 = [0,1,0], x_2 = [0,0,1]
# y_0_true = [0,1,0], y_1_true = [0,0,1], y_2_true = [1,0,0]

inputs_indices = [0, 1, 2, 3, 4]
targets_indices = [1, 2, 0, 4, 3] 

# 원-핫 인코딩된 시퀀스 생성
inputs_sequence_onehot = [np.eye(vocab_size)[i] for i in inputs_indices]
targets_sequence_onehot = [np.eye(vocab_size)[i] for i in targets_indices]

# RNN 모델 인스턴스 생성
rnn = SimpleRNNNumpy(vocab_size, hidden_size, output_size, learning_rate)

print("--- 학습 시작 ---")
print(f"순환 가중치 Whh (초기 상태 일부):\n{rnn.Whh[:2, :2]}\n") # Whh의 일부 값만 출력

# 학습 루프
for epoch in range(epochs):
    # 1. 순전파
    predictions_sequence = rnn.forward(inputs_sequence_onehot)
    
    # 2. 역전파 (BPTT)
    total_loss, dWxh, dWhh, dWhy, dbh, dby = rnn.backward(inputs_sequence_onehot, targets_sequence_onehot, predictions_sequence)
    
    # 3. 가중치 업데이트
    rnn.update_weights(dWxh, dWhh, dWhy, dbh, dby)
    
    # 10 에포크마다 손실 및 Whh의 변화 출력
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss:.4f}")
        print(f"  순환 가중치 Whh (일부 업데이트):\n{rnn.Whh[:2, :2]}\n")

print("--- 학습 종료 ---")
print(f"순환 가중치 Whh (최종 상태 일부):\n{rnn.Whh[:2, :2]}")

# 학습 후 예측 테스트
print("\n--- 학습 후 간단 예측 테스트 ---")
# 테스트 시에는 이전 은닉 상태를 수동으로 관리해야 합니다.
current_h_state = np.zeros((1, hidden_size)) # 초기 은닉 상태
for i in range(len(inputs_sequence_onehot)):
    xt_test = np.array(inputs_sequence_onehot[i]).reshape(1, -1)
    ht_linear_test = np.dot(xt_test, rnn.Wxh) + np.dot(current_h_state, rnn.Whh) + rnn.bh
    ht_test = tanh(ht_linear_test) # 현재 은닉 상태
    yt_pred_test = np.dot(ht_test, rnn.Why) + rnn.by # 현재 예측
    
    predicted_next_symbol_index = np.argmax(yt_pred_test) # 가장 확률 높은 다음 심볼 인덱스
    
    print(f"입력 심볼 인덱스: {inputs_indices[i]}, "
          f"모델이 예측한 다음 심볼 인덱스: {predicted_next_symbol_index} (실제 다음 심볼: {targets_indices[i]})")
    
    current_h_state = ht_test # 다음 예측을 위해 현재 은닉 상태를 업데이트

--- 학습 시작 ---
순환 가중치 Whh (초기 상태 일부):
[[-0.01696312 -0.00205441]
 [ 0.00560655 -0.00546124]]

Epoch 10/100, Loss: 0.3997
  순환 가중치 Whh (일부 업데이트):
[[-0.01683986 -0.00204966]
 [ 0.00539843 -0.00541921]]

Epoch 20/100, Loss: 0.3981
  순환 가중치 Whh (일부 업데이트):
[[-0.01643088 -0.00177034]
 [ 0.00482824 -0.00577398]]

Epoch 30/100, Loss: 0.3873
  순환 가중치 Whh (일부 업데이트):
[[-0.01439533  0.00059348]
 [ 0.0019874  -0.00886318]]

Epoch 40/100, Loss: 0.3296
  순환 가중치 Whh (일부 업데이트):
[[-0.00315135  0.01775442]
 [-0.01356599 -0.02640534]]

Epoch 50/100, Loss: 0.1830
  순환 가중치 Whh (일부 업데이트):
[[ 0.03940565  0.10426338]
 [-0.07210296 -0.0615634 ]]

Epoch 60/100, Loss: 0.1021
  순환 가중치 Whh (일부 업데이트):
[[ 0.0728035   0.20560463]
 [-0.13044675 -0.07048593]]

Epoch 70/100, Loss: 0.0729
  순환 가중치 Whh (일부 업데이트):
[[ 0.09163396  0.23102933]
 [-0.15657206 -0.08272594]]

Epoch 80/100, Loss: 0.0124
  순환 가중치 Whh (일부 업데이트):
[[ 0.10846486  0.23818541]
 [-0.18741785 -0.09945152]]

Epoch 90/100, Loss: 0.0001
  순환 가중치 Whh (일부 업데이트):
